In [4]:
import numpy as np

def create_input_space(categorical_data, continuous_data):
    """
    Creates a simple structured input space.
    
    Parameters:
    - categorical_data (dict): Keys are categorical variable names, values are lists of possible categories.
    - continuous_data (dict): Keys are variable names, values are (min, max) tuples.
    
    Returns:
    - bounds (np.array): Bounds for continuous variables.
    - labeled_input_space (list): List of input labels in the format h1, h2, x1, x2...
    """
    labeled_input_space = []
    continuous_count = 1
    bounds = []
    
    # Add labels for categorical variables (just integers for simplicity)
    categorical_count = 1
    for cat_var, categories in categorical_data.items():
        labeled_input_space.append(f"h{categorical_count}")
        categorical_count += 1
    
    # Add labels for continuous variables and bounds
    for cont_var, (min_val, max_val) in continuous_data.items():
        bounds.append([min_val, max_val])
        labeled_input_space.append(f"x{continuous_count}")
        continuous_count += 1
    
  
    return np.array(bounds), labeled_input_space

# Example case
categorical_data = {
    "Category1": [0, 1],  # 2 choices
    "Category2": [0, 1, 2]  # 3 choices
}
continuous_data = {
    "Var1": (0, 1),
    "Var2": (0, 1),
    "Var3": (0, 1),
    "Var4": (0, 1)
}

bounds, labeled_input_space = create_input_space(categorical_data, continuous_data)
print("Continuous Bounds:", bounds)
print("Labeled Input Space:", labeled_input_space)


Continuous Bounds: [[0 1]
 [0 1]
 [0 1]
 [0 1]]
Labeled Input Space: ['h1', 'h2', 'x1', 'x2', 'x3', 'x4']


In [7]:
import torch
import numpy as np
from botorch.models import SingleTaskGP
from gpytorch.likelihoods import GaussianLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf

# Define the EXP3 algorithm for categorical optimization
class EXP3:
    def __init__(self, n_arms, gamma=0.1):
        self.n_arms = n_arms
        self.gamma = gamma
        self.weights = np.ones(n_arms)  # Initialize the weights
        self.total_reward = 0

    def select_arm(self):
        # Compute the probability distribution over arms
        prob_dist = (1 - self.gamma) * (self.weights / np.sum(self.weights)) + (self.gamma / self.n_arms)
        # Select an arm based on the probability distribution
        return np.random.choice(self.n_arms, p=prob_dist)

    def update(self, arm, reward):
        # Update the weights based on the observed reward
        self.total_reward += reward
        self.weights[arm] *= np.exp(self.gamma * reward / (self.n_arms * np.sum(self.weights)))

# Example: Define categorical and continuous input spaces
categorical_data = {
    "Category1": [0, 1],  # 2 choices
    "Category2": [0, 1, 2]  # 3 choices
}
continuous_data = {
    "Var1": (0, 1),
    "Var2": (0, 1),
    "Var3": (0, 1),
    "Var4": (0, 1)
}

# Set up the GP model for continuous variables
def create_gp_model(train_x, train_y):
    likelihood = GaussianLikelihood()
    model = SingleTaskGP(train_x, train_y.unsqueeze(-1), likelihood=likelihood)
    return model

# Define the acquisition function
def optimize_continuous(model, bounds):
    acq_func = UpperConfidenceBound(model, beta=2.0)
    candidates, _ = optimize_acqf(acq_func, bounds=bounds, q=1, num_restarts=5, raw_samples=20)
    return candidates

# Initialize the EXP3 model for categorical optimization
n_arms = 3  # Example: 3 categorical choices
exp3 = EXP3(n_arms)

# Example: Initial continuous points for training the GP
train_x = torch.tensor([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8], [0.9, 0.8, 0.7, 0.6]], dtype=torch.float64)
train_y = torch.tensor([0.5, 0.8, 0.6], dtype=torch.float64)  # Example target values

# Train the GP model
model = create_gp_model(train_x, train_y)

# Get the bounds for the continuous variables
bounds = torch.tensor([[0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0]])

# Simulate the optimization process
# Select a categorical arm (e.g., choose a category for Category1 and Category2)
selected_arm = exp3.select_arm()
print(f"Selected Categorical Arm: {selected_arm}")

# Optimize the continuous space using the GP model
continuous_candidate = optimize_continuous(model, bounds)
print(f"Suggested Continuous Candidate: {continuous_candidate}")

# Combine the results: The final input zt = [ht, xt]

# First reshape the selected_arm to match the continuous candidate shape
selected_arm_tensor = torch.tensor([selected_arm]).unsqueeze(0) #Shape [1,1] 
continuous_candidate_tensor = continuous_candidate # Shape [1,4]

# Here we assume `ht` is the selected categorical arm, and `xt` is the optimized continuous candidate
zt = torch.cat((selected_arm_tensor, continuous_candidate_tensor), dim=-1)  #BECAREFUL HERE AS THE INTEGER CATEGORIES WILL CONVERT TO FLOATS
print(f"Final input zt: {zt}")


Selected Categorical Arm: 2
Suggested Continuous Candidate: tensor([[0.2908, 0.6281, 0.9655, 1.0000]])
Final input zt: tensor([[2.0000, 0.2908, 0.6281, 0.9655, 1.0000]])


In [8]:
import torch
from gpytorch.kernels import Kernel, MaternKernel, ScaleKernel
from gpytorch.constraints import Interval

# Define a categorical kernel using a similarity matrix
class CategoricalKernel(Kernel):
    def __init__(self, similarity_matrix, **kwargs):
        super().__init__(**kwargs)
        self.similarity_matrix = similarity_matrix

    def forward(self, x1, x2, diag=False, **params):
        x1, x2 = x1.long().squeeze(), x2.long().squeeze()
        return self.similarity_matrix[x1, x2]

# Define the hybrid kernel (continuous + categorical)
class HybridKernel(Kernel):
    def __init__(self, cont_kernel, cat_kernel, **kwargs):
        super().__init__(**kwargs)
        self.cont_kernel = cont_kernel
        self.cat_kernel = cat_kernel
        self.lambda_param = torch.nn.Parameter(torch.tensor(0.5, dtype=torch.float64), requires_grad=True)
        self.register_constraint("lambda_param", Interval(0.0, 1.0))

    def forward(self, x1, x2, diag=False, **params):
        x1_cat, x1_cont = x1[:, 0], x1[:, 1:]
        x2_cat, x2_cont = x2[:, 0], x2[:, 1:]
        
        k_cat = self.cat_kernel(x1_cat, x2_cat, diag=diag, **params)
        k_cont = self.cont_kernel(x1_cont, x2_cont, diag=diag, **params)
        
        return self.lambda_param * k_cont + (1 - self.lambda_param) * k_cat

# Define a similarity matrix for categorical variables (example with 3 categories)
similarity_matrix = torch.tensor([
    [1.0, 0.5, 0.2],
    [0.5, 1.0, 0.3],
    [0.2, 0.3, 1.0],
], dtype=torch.float64)

# Instantiate the kernels
cat_kernel = CategoricalKernel(similarity_matrix)
cont_kernel = ScaleKernel(MaternKernel(nu=2.5))

# Create the hybrid kernel
hybrid_kernel = HybridKernel(cont_kernel, cat_kernel)

# Apply hybrid kernel to GP model
model.covar_module = hybrid_kernel


In [ ]:
# below is some chatgpt shit. Dont take it for granted!!!@!!!!!!!

In [10]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from botorch.models import SingleTaskGP
from gpytorch.likelihoods import GaussianLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf

# Simulated objective function (for testing optimization)
def objective_function(h1, h2, x1, x2, x3, x4):
    return (h1 + h2) * torch.sin(3 * x1) + 0.5 * torch.cos(2 * x2) + x3**2 - 0.3 * x4

# Categorical (integer encoded) and continuous variables
categorical_choices = torch.tensor([
    [0, 0], [0, 1], [0, 2],  # Category1 (h1) & Category2 (h2)
    [1, 0], [1, 1], [1, 2]
], dtype=torch.float64)

continuous_bounds = torch.tensor([[0, 0, 0, 0], [1, 1, 1, 1]], dtype=torch.float64)

# Generate initial training data (random sampling)
n_initial = 6
train_x_cat = categorical_choices[torch.randint(0, len(categorical_choices), (n_initial,))]
train_x_cont = torch.rand((n_initial, 4), dtype=torch.float64)
train_x = torch.cat([train_x_cat, train_x_cont], dim=1)

train_y = torch.tensor([
    objective_function(*point) for point in train_x
], dtype=torch.float64).unsqueeze(-1)  # Ensure shape is (N,1)


In [11]:
# Create and train the GP model
likelihood = GaussianLikelihood()
model = SingleTaskGP(train_x, train_y, likelihood=likelihood)
model.covar_module = hybrid_kernel  # Use hybrid kernel

# Training loop (simple)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

for i in range(50):  # Train for 50 iterations
    optimizer.zero_grad()
    output = model(train_x)
    loss = -mll(output, train_y)
    loss.backward()
    optimizer.step()

print("Model Training Completed!")


c:\Users\uqkmuroi\gitcode\bayesmediaopt\venv\Lib\site-packages\botorch\models\utils\assorted.py:264: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(


NameError: name 'gpytorch' is not defined

In [12]:
# EXP3 for categorical choices
exp3 = EXP3(len(categorical_choices))

# Bayesian Optimization Loop
num_iterations = 10
best_observed = []
for i in range(num_iterations):
    # Select categorical variables using EXP3
    selected_cat_idx = exp3.select_arm()
    selected_cat = categorical_choices[selected_cat_idx]

    # Optimize continuous variables using BO
    acq_func = UpperConfidenceBound(model, beta=2.0)
    candidates, _ = optimize_acqf(acq_func, bounds=continuous_bounds, q=1, num_restarts=5, raw_samples=20)

    # Form final candidate [h1, h2, x1, x2, x3, x4]
    zt = torch.cat((selected_cat, candidates), dim=-1)

    # Evaluate function
    new_y = objective_function(*zt).unsqueeze(-1)

    # Update dataset
    train_x = torch.cat([train_x, zt.unsqueeze(0)], dim=0)
    train_y = torch.cat([train_y, new_y.unsqueeze(0)], dim=0)

    # Update GP model
    model.set_train_data(train_x, train_y, strict=False)

    # Store best result
    best_observed.append(train_y.max().item())

    print(f"Iteration {i+1}: Selected {zt}, Function Value = {new_y.item()}")

# Plot best observed function values
plt.plot(range(1, num_iterations+1), best_observed, marker='o')
plt.xlabel("Iteration")
plt.ylabel("Best Observed Value")
plt.title("Bayesian Optimization Progress")
plt.grid()
plt.show()


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 6 but got size 4 for tensor number 1 in the list.

4️⃣ Expected Outputs
Selected categorical variables (e.g., [0,1], [1,2], etc.)

Optimized continuous values (e.g., [0.7, 0.5, 0.9, 0.3])

Best observed function values plotted over iterations

🎯 Summary
✅ GP + Hybrid Kernel → Handles categorical & continuous spaces
✅ EXP3 → Optimizes categorical variables
✅ Bayesian Optimization → Optimizes continuous variables
✅ Visual Output → Convergence graph showing BO progress

Let me know if you need any refinements! 🚀